# Content

* ### 1 - Environment settings
* ### 2 - Load data
* ### 3 - Data preparation
* ### 4 - Graph creation
* ### 5 - Routing algorithm
  * ### 5.1 - Helper methods
  * ### 5.2 - Algorithm
* ### 6 - Visualization
    * ### 6.1 - Helper methods
    * ### 6.2 - Visualization interface

# 1 - Environment settings 

### Spark session

In [1]:
import os
import pwd
import numpy as np
import sys

from pyspark.sql import SparkSession
from random import randrange
import pyspark.sql.functions as F
#np.bool = np.bool_


username = pwd.getpwuid(os.getuid()).pw_name
hadoopFS=os.getenv('HADOOP_FS', None)
groupName = 'L1'

print(os.getenv('SPARK_HOME'))
print(f"hadoopFSs={hadoopFS}")
print(f"username={username}")
print(f"group={groupName}")

/opt/spark
hadoopFSs=hdfs://iccluster059.iccluster.epfl.ch:9000
username=valat
group=L1


In [2]:
spark = SparkSession\
            .builder\
            .appName(pwd.getpwuid(os.getuid()).pw_name)\
            .config('spark.ui.port', randrange(4040, 4440, 5))\
            .config("spark.executorEnv.PYTHONPATH", ":".join(sys.path)) \
            .config('spark.jars', f'{hadoopFS}/data/com-490/jars/iceberg-spark-runtime-3.5_2.13-1.6.1.jar')\
            .config('spark.sql.extensions', 'org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions')\
            .config('spark.sql.catalog.iceberg', 'org.apache.iceberg.spark.SparkCatalog')\
            .config('spark.sql.catalog.iceberg.type', 'hadoop')\
            .config('spark.sql.catalog.iceberg.warehouse', f'{hadoopFS}/data/com-490/iceberg/')\
            .config('spark.sql.catalog.spark_catalog', 'org.apache.iceberg.spark.SparkSessionCatalog')\
            .config('spark.sql.catalog.spark_catalog.type', 'hadoop')\
            .config('spark.sql.catalog.spark_catalog.warehouse', f'{hadoopFS}/user/{username}/assignment-3/warehouse')\
            .config("spark.sql.warehouse.dir", f'{hadoopFS}/user/{username}/assignment-3/spark/warehouse')\
            .config('spark.eventLog.gcMetrics.youngGenerationGarbageCollectors', 'G1 Young Generation')\
            .config("spark.executor.memory", "6g")\
            .config("spark.executor.cores", "4")\
            .config("spark.executor.instances", "4")\
            .master('yarn')\
            .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/06/01 09:32:34 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


### Imports

In [3]:
groupName='L1'

In [4]:
import networkx as nx

In [5]:
import os
import warnings

warnings.simplefilter(action='ignore', category=UserWarning)
warnings.filterwarnings("ignore", category=UserWarning, message="pandas only supports SQLAlchemy connectable .*")

In [6]:
import base64 as b64
import json
import time
import re

def getUsername():
    payload = os.environ.get('EPFL_COM490_TOKEN').split('.')[1]
    payload=payload+'=' * (4 - len(payload) % 4)
    obj = json.loads(b64.urlsafe_b64decode(payload))
    if (time.time() > int(obj.get('exp')) - 3600):
        raise Exception('Your credentials have expired, please restart your Jupyter Hub server:'
                        'File>Hub Control Panel, Stop My Server, Start My Server.')
    time_left = int((obj.get('exp') - time.time())/3600)
    return obj.get('sub'), time_left

In [7]:
username, validity_h = getUsername()
hadoopFS = os.environ.get('HADOOP_FS')
namespace = 'iceberg.' + username
sharedNS = 'iceberg.com490_iceberg'

if not re.search('[A-Z][0-9]', groupName):
    raise Exception('Invalid group name {groupName}')

print(f"you are: {username}")
print(f"credentials validity: {validity_h} hours left.")
print(f"shared namespace is: {sharedNS}")
print(f"your namespace is: {namespace}")
print(f"your group is: {groupName}")

you are: valat
credentials validity: 166 hours left.
shared namespace is: iceberg.com490_iceberg
your namespace is: iceberg.valat
your group is: L1


In [8]:
import trino
from contextlib import closing
from urllib.parse import urlparse
from trino.dbapi import connect
from trino.auth import BasicAuthentication, JWTAuthentication

trinoAuth = JWTAuthentication(os.environ.get('EPFL_COM490_TOKEN'))
trinoUrl  = urlparse(os.environ.get('TRINO_URL'))
Query=[]

print(f"Warehouse URL: {trinoUrl.scheme}://{trinoUrl.hostname}:{trinoUrl.port}/")

conn = connect(
    host=trinoUrl.hostname,
    port=trinoUrl.port,
    auth=trinoAuth,
    http_scheme=trinoUrl.scheme,
    verify=True
)

print('Connected!')

Warehouse URL: https://iccluster028.iccluster.epfl.ch:8443/
Connected!


In [9]:
import pandas as pd

# 2 - Load data

### Delay prediction model

In [10]:
from pyspark.ml.pipeline import PipelineModel

model_path = f"{hadoopFS}/user/com-490/group/{groupName}/delay_model"
delay_model = PipelineModel.load(model_path)

### Historical average delay

In [11]:
avg_delay_by_hour = spark.read.csv(f"{hadoopFS}/user/com-490/group/{groupName}/avg_delay_by_hour.csv", header=True, inferSchema=True)

In [12]:
avg_delay_by_bpuic = spark.read.csv(f"{hadoopFS}/user/com-490/group/{groupName}/avg_delay_by_bpuic.csv", header=True, inferSchema=True)

### Stop times
It contains the stop times and weekdays of trips (trip_id) servicing stops found previously in the selected region(s).

In [13]:
path_stop_stimes = f"{hadoopFS}/user/com-490/group/{groupName}/sbb_stop_times_selected_regions/data"
spark_sbb_stop_times_region = spark.read.parquet(path_stop_stimes)

sbb_stop_times_region = spark_sbb_stop_times_region.toPandas()

### Stops

In [14]:
path_stops = f"{hadoopFS}/user/com-490/group/{groupName}/sbb_stops_selected_regions/data"
spark_sbb_stops_region = spark.read.parquet(path_stops)
sbb_stops_region = spark_sbb_stops_region.toPandas()

### Transfers

Contains the stop times between two direct transfers (same lat/lon) 

In [15]:
query_df = f"""
    SELECT *
    FROM {sharedNS}.sbb_transfers
"""
transfers_df = pd.read_sql(query_df, conn)

### Walking distance stops
Contains stops within 500m of each other, as directed pairs

In [16]:
path_stops_to_stops = f"{hadoopFS}/user/com-490/group/{groupName}/sbb_stops_to_stops_selected_regions/data"
spark_sbb_stops_to_stops_region = spark.read.parquet(path_stops_to_stops)
sbb_stops_to_stops_df = spark_sbb_stops_to_stops_region.toPandas()

### Routes and trips 
Used in the visualizations to have the names of routes. 
This takes a long time to load.

In [17]:
query_df = f"""
    SELECT *
    FROM {sharedNS}.sbb_routes
    LIMIT 5
"""
pd.read_sql(query_df, conn)

,route_id,agency_id,route_short_name,route_long_name,route_desc,route_type,pub_date
0,91-10-A-j21-1,37,10,,T,900,2021-10-20
1,91-10-B-j21-1,06____,S10,,S,109,2021-10-20
2,91-10-C-j21-1,11,RE10,,RE,106,2021-10-20
3,91-10-D-j21-1,11,S10,,S,109,2021-10-20
4,91-10-F-j21-1,78,S10,,S,109,2021-10-20


In [18]:
query_df = f"""
    SELECT distinct t.trip_id, r.route_id, r.route_short_name, r.route_desc
    FROM {sharedNS}.sbb_trips t
    INNER JOIN {sharedNS}.sbb_routes r on r.route_id = t.route_id
    """

trip_names_df = pd.read_sql(query_df, conn)
trip_names_df.head()

,trip_id,route_id,route_short_name,route_desc
0,47.TA.92-690-j21-1.21.R,92-690-j21-1,690,Bus
1,56.TA.92-690-j21-1.25.R,92-690-j21-1,690,Bus
2,63.TA.92-691-j21-1.5.R,92-691-j21-1,691,B
3,32.TA.92-693-j21-1.3.R,92-693-j21-1,693,Bus
4,36.TA.92-693-j21-1.3.R,92-693-j21-1,693,B


# 3 - Data preparation 

### Filter the stops
We only want to consider journeys at reasonable hours of the day, and on a typical business day, and assuming a recent schedule. So, we only keep stops that are : 
- from the selected region(s)
- operating on a weekday
- between EARLIEST_TIME and LATEST_TIME 

In [19]:
EARLIEST_TIME = '08:00:00'
LATEST_TIME = '20:00:00'

weekdays = ['monday', 'tuesday', 'wednesday', 'thursday', 'friday']
sbb_stops_region_filtered = sbb_stop_times_region[sbb_stop_times_region[weekdays].any(axis=1)] \
    [
        (sbb_stop_times_region['departure_time'] >= EARLIEST_TIME) &
        (sbb_stop_times_region['departure_time'] <= LATEST_TIME) &
        (sbb_stop_times_region['arrival_time'] >= EARLIEST_TIME) &
        (sbb_stop_times_region['arrival_time'] <= LATEST_TIME) 
    ]

In [20]:
sbb_stops_region_filtered = sbb_stops_region_filtered.merge(sbb_stops_region, left_on='stop_id', right_on= 'stop_id_cleaned', how='left').merge(trip_names_df, on = 'trip_id', how = 'left')

### Filter trip_names_df 
We only keep trips from our selected region(s). 

In [21]:
region_trips = set(sbb_stops_region_filtered['trip_id'])
trip_names_df = trip_names_df[trip_names_df['trip_id'].isin(region_trips)]

In [23]:
sbb_stops_region_filtered.head(3)

,trip_id,stop_id,departure_time,arrival_time,monday,tuesday,wednesday,thursday,friday,saturday,sunday,stop_id_cleaned,stop_name,stop_lat,stop_lon,route_id,route_short_name,route_desc
0,31.TA.96-240-j24-1.2.R,8570064,19:50:00,19:50:00,True,True,True,True,True,True,True,8570064,"Cheseaux-sur-L., Pâquis",46.586866711232,6.603785149067,96-240-j24-1,425,B
1,31.TA.96-240-j24-1.2.R,8570063,19:49:00,19:49:00,True,True,True,True,True,True,True,8570063,"Cheseaux-sur-L., gare",46.584168733670,6.605114655688,96-240-j24-1,425,B
2,95.TA.92-54-B-j24-1.4.R,8593816,18:35:00,18:35:00,True,True,True,True,True,True,True,8593816,"Crissier, Marcolet",46.546932815805,6.574868380067,92-54-B-j24-1,54,B


### List of unique stop names
To use later for visualization

In [25]:
unique_stop_names = sbb_stops_region_filtered['stop_name'].drop_duplicates().tolist()
unique_stop_names = sorted(unique_stop_names)

### Filter the transfers
- Keep only the transfers that are within the selected region. 
- Do this by keeping only (from_stop_id, to_stop_id) in sbb_stops_region_filtered.stop_id
- keep only the most recent pub_date per (from_stop_id, to_stop_id) pair in transfers_df_filtered
- convert to minutes

In [26]:
# Get the set of valid stop_ids
valid_stops = set(sbb_stops_region_filtered['stop_id'])

# Filter sbb_transfers
transfers_df_filtered = transfers_df[
    transfers_df['from_stop_id'].isin(valid_stops) &
    transfers_df['to_stop_id'].isin(valid_stops)
]

# Find the index of the most recent pub_date per (from_stop_id, to_stop_id) pair
idx = transfers_df_filtered.groupby(['from_stop_id', 'to_stop_id'])['pub_date'].idxmax()
transfers_df_filtered = transfers_df_filtered.loc[idx].reset_index(drop=True)

# Convert to min
transfers_df_filtered['min_transfer_time'] = transfers_df_filtered['min_transfer_time'] / 60

### Create the walk transfers
- We use table sbb_stops_to_stops_df that contains all stops that are within 500m walking distance between each other.
- Add a column _distance_ that estimates walking time between points
- I don't remove stop pairs from sbb_stops_to_stops_df_filtered that are already present in transfers_df_filtered, so there may be duplicate edges for walking transfers, but the algo will take the min anyways


In [27]:
def req_walking_time(distance):
    # In minutes - to be consistent with other times
    # For walking speed we use assumption of 50m/min
    # We don't add an additional 2min for transfer within same location
    walking_time = distance / 50 
    
    return walking_time

sbb_stops_to_stops_df_filtered = sbb_stops_to_stops_df[
    sbb_stops_to_stops_df['stop_id_a'].isin(valid_stops) &
    sbb_stops_to_stops_df['stop_id_a'].isin(valid_stops)
]

sbb_stops_to_stops_df_filtered['duration'] = req_walking_time(sbb_stops_to_stops_df_filtered['distance'])

/tmp/ipykernel_1453504/3885168292.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sbb_stops_to_stops_df_filtered['duration'] = req_walking_time(sbb_stops_to_stops_df_filtered['distance'])


In [28]:
sbb_stops_to_stops_df_filtered[sbb_stops_to_stops_df_filtered['stop_id_a'] == '8592086']

,stop_id_a,stop_id_b,distance,duration
1894,8592086,8595933,478.711259,9.574225
2147,8592086,8501075,276.230739,5.524615
2819,8592086,8591986,113.884055,2.277681
2878,8592086,8592061,280.372377,5.607448


# 4 - Graph creation

In [30]:
from datetime import time, datetime
from queue import PriorityQueue

'''
Convert a given time t to seconds from midnight.
Can accept 3 different forma tof time : string ('HH:MM:SS'), datetime and time.
'''
def time_to_minutes(t):
    # For 'HH:MM:SS' format (string)
    if isinstance(t, str):
        h, m, s = map(int, t.split(':'))
    # For time and datetime format
    elif isinstance(t, datetime.time) or isinstance(t, datetime.datetime):
        h, m, s = t.hour, t.minute, t.second
    else:
        raise ValueError("Input should be a string, datetime.time, or datetime.datetime object.")
    
    return h * 60 + m + s / 60

sbb_stops_region_filtered['dep_minutes'] = sbb_stops_region_filtered['departure_time'].apply(time_to_minutes)
sbb_stops_region_filtered['arr_minutes'] = sbb_stops_region_filtered['arrival_time'].apply(time_to_minutes)

sbb_stops_region_filtered = sbb_stops_region_filtered.sort_values(by=['trip_id', 'dep_minutes'])
sbb_stops_region_filtered.head(3)

,trip_id,stop_id,departure_time,arrival_time,monday,tuesday,wednesday,thursday,friday,saturday,sunday,stop_id_cleaned,stop_name,stop_lat,stop_lon,route_id,route_short_name,route_desc,dep_minutes,arr_minutes
137476,1.TA.91-m2-j24-1.1.H,8592050,16:59:00,16:59:00,True,True,True,True,True,False,False,8592050,"Lausanne, gare",46.517201094384,6.632118013132,91-m2-j24-1,m2,M,1019.0,1019.0
137477,1.TA.91-m2-j24-1.1.H,8592050,16:59:00,16:59:00,True,True,True,True,True,False,False,8592050,"Lausanne, gare",46.517602899357,6.629656629253,91-m2-j24-1,m2,M,1019.0,1019.0
137478,1.TA.91-m2-j24-1.1.H,8592050,16:59:00,16:59:00,True,True,True,True,True,False,False,8592050,"Lausanne, gare",46.517201094384,6.632118013132,91-m2-j24-1,m2,M,1019.0,1019.0


### Build the graph 

In [31]:
graph = {}

# Group by trip_id and build the graph
for trip_id, group in sbb_stops_region_filtered.groupby('trip_id'):
    # for each trip, get the stops and their characteristics
    trip_stops = group[['stop_id', 'dep_minutes', 'arr_minutes']].values

    # construct the edges as described above stop i -> list of (dep time from i, arr time to j, stop j)
    for i in range(len(trip_stops)-1):
        from_stop, dep_time_min, _ = trip_stops[i]
        to_stop, _, arr_time_min = trip_stops[i + 1]

        # Store edge in graph
        if from_stop not in graph:
            graph[from_stop] = []
        graph[from_stop].append((dep_time_min, arr_time_min, to_stop, trip_id))

### Turn the graph into a network graph, and add more edges

In [32]:
# Create a directed graph
G = nx.MultiDiGraph()

# 1. Add nodes from df
stop_ids = sbb_stops_region_filtered['stop_id'].unique()
G.add_nodes_from(stop_ids)

# 2. Add edges from the `graph` dictionary
# graph['from_stop_id'] = [(dep_time_min, arr_time_min, to_stop_id, trip_id), ...]
for from_stop_id, edges in graph.items():
    for dep_time_min, arr_time_min, to_stop_id, trip_id in edges:
        G.add_edge(
            from_stop_id,
            to_stop_id,
            dep_time_min=dep_time_min,
            arr_time_min=arr_time_min,
            trip_id=trip_id,
            edge_type='transit'
        )
        
# 3. Add transfer edges (e.g. walking connections) between stops, separate from vehicle-based edges.
for _, row in transfers_df_filtered.iterrows():
    from_stop = row['from_stop_id']
    to_stop = row['to_stop_id']
    transfer_type = row['transfer_type']
    min_transfer_time = row['min_transfer_time']
    pub_date = row.get('pub_date')  # optional

    # Add the transfer edge with attributes
    G.add_edge(
        from_stop,
        to_stop,
        transfer_type=transfer_type,
        min_transfer_time=min_transfer_time,
        pub_date=pub_date,
        edge_type='transfer'  # distinguish from vehicle-based edges
    )

# 4. Add walking edges between stops that are < 500m from each other.
for _, row in sbb_stops_to_stops_df_filtered.iterrows():
    from_stop = row['stop_id_a']
    to_stop = row['stop_id_b']
    distance = row['distance']
    duration = row['duration']

    G.add_edge(
        from_stop,
        to_stop,
        distance=distance,
        duration=duration,
        edge_type='walking'
    )

# Reverse the graph to make it easier to traverse backwards
G = G.reverse(copy=True)

# 5 - Routing algorithm 

## 5.1 - Helper methods 

In [33]:
from heapq import heappush, heappop
from itertools import count
import datetime

from pyspark.ml.feature import VectorAssembler
from pyspark.sql import SparkSession
from pyspark.sql.functions import lit
import pyspark.sql.functions as F
from pyspark.sql.functions import udf
from pyspark.sql.types import DoubleType

In [34]:
''' 
Methods to get score of a path. Score defined by departure time.
'''
def score_path_by_latest_departure(path):
    # In reversed route, the first node is destination, last is origin
    dep_time = path[-1]['dep_time_min']
    return dep_time

In [35]:
'''
Given a leg (= from node, to node, departute time, arrival time, trip_id and type of trip):
Build the corresponding feature vector to be given to the delay model
The vector must have the following columns :  rain (1 if rainy day, 0 otherwise), hour, day_of_week, avg_delay_hour, avg_delay_bpuic
'''
def build_features_from_leg(leg, weather_condition, date_of_the_day):
    rain = 0 if weather_condition=='Clear' else 1
    hour = date_of_the_day.hour
    
    day_of_week = date_of_the_day.weekday() # Monday = 0, Sunday = 6

    # Get avg_delay_hour for the current hour
    filtered_data = avg_delay_by_hour[avg_delay_by_hour["hour"] == hour]
    avg_delay_hour_row = filtered_data.select("avg_delay_hour").first()
    avg_delay_hour = avg_delay_hour_row[0] if avg_delay_hour_row else 0.0  # Default to 0 if no data

    # Get avg_delay_bpuic for the destination stop
    bpuic = leg['to']
    filtered_data = avg_delay_by_bpuic[avg_delay_by_bpuic["bpuic"] == bpuic]
    avg_delay_bpuic_row = filtered_data.select("avg_delay_bpuic").first()
    avg_delay_bpuic = avg_delay_bpuic_row[0] if avg_delay_bpuic_row else 0.0  # Default to 0 if no data


    # Check, in case of nan (should not have but if does, put to 0)
    if pd.isna(avg_delay_hour) or pd.isna(avg_delay_bpuic):
        print("Invalid data found: avg_delay_hour or avg_delay_bpuic is NaN or None.")
        avg_delay_hour = 0.0
        avg_delay_bpuic = 0.0

    # Convert the features to a list of values
    feature_list = [
        rain,
        hour,
        day_of_week,
        avg_delay_hour,
        avg_delay_bpuic
    ]
    return feature_list


def get_confidence(path, model, weather_condition, date_of_the_day):
    """
    Get the confidence level for a given path by evaluating each leg (if it's a transit leg).
   
    Parameters:
    - path (list): List of leg dictionaries representing the path.
    - model (PipelineModel): The trained model for prediction.
    - weather_condition (str): Weather condition ('Clear' or 'Rain').
    - date_of_the_day (datetime): The date of the day for feature extraction.
    
    Returns:
    - float: The overall confidence for the path.
    """
    
    feature_list = []
    
    for leg in path:
        if leg['type'] == 'transit':
            # Get leg's feature and append it to feature_list
            feature_vector = build_features_from_leg(leg, weather_condition, date_of_the_day) 
            feature_list.append(feature_vector)
    
    if not feature_list:
        return 1.0  # Assume 100% confidence if no transit legs
    
    # Now create a DataFrame for all features
    features_df = spark.createDataFrame(feature_list, ['rain', 'hour', 'day_of_week', 'avg_delay_hour', 'avg_delay_bpuic'])

    # Use VectorAssembler to convert to vector format for ML
    assembler = VectorAssembler(inputCols=['rain', 'hour', 'day_of_week', 'avg_delay_hour', 'avg_delay_bpuic'], outputCol="features")
    feature_vector_df = assembler.transform(features_df)

    # Transform all features at once
    predictions = model.transform(feature_vector_df)

    # Extract probabilities for class 1 (probability of delay) for all legs at once
    get_prob_class1 = udf(lambda prob: float(prob[1]), DoubleType())
    predictions_with_probs = predictions.withColumn("probability_class_1", get_prob_class1("probability"))
    # Extract the confidence for each leg and calculate overall confidence
    confidences = predictions_with_probs.select("probability_class_1").rdd.flatMap(lambda x: x).collect()

    conf = np.prod(confidences)
    conf = 1 - conf # get proba of confidence (not delayed = 1 - delayed)
    conf = conf*100 # put it in %, as it will be compared with Q = 80 (for example)
    return conf 

In [36]:
'''Helper class to store top k paths'''
class TopKPathsPerNode:
    '''
    k: how many best paths to keep per node
    score_fn: function that takes a path and returns a number (higher = better)
    confidence_fn: function that takes a path and returns a float in [0,1]
    Q: the minimum acceptable confidence threshold
    rain : boolean, True if the day considered is a rainy day (used for the delay prediction model)
    '''
    def __init__(self, k, score_fn, confidence_fn, Q, delay_model, weather_condition, date_of_the_day):
        self.k = k
        self.score_fn = score_fn
        self.confidence_fn = confidence_fn
        self.Q = Q
        self.weather_condition = weather_condition
        self.delay_model = delay_model
        self.date_of_the_day = date_of_the_day
        self.paths = {}  
        # Dictionnary paths has structure : node_id -> list of (score, path, key, confidence) ; key is used to check for similar paths. 
        # For a path x : {'from': 'A', 'to': 'B', 'dep_time_min': 600, 'arr_time_min': 610, 'trip_id': 'X'},{'from': 'B', 'to': 'C', 'dep_time_min': 615, 'arr_time_min': 630, 'trip_id': 'Y'}
        # -> the key of path x is (('A','B',600, 610), ('B','C',615, 630))


    '''
    Given a node and a path, add this path to the paths directory at entry node. Add the path only if : 
        - Not similar to a path already stored. Similar is defined by same key (i.e., if has the same stops and same departure/arrival times)
        - Its confidence level is above Q
    Then, re sort the list of path related to the node, based on score_fn, and keep only the k tops
    '''
    def add_path(self, node, path):
        key_new = tuple((leg['from'], leg['to'], leg['dep_time_min'], leg['arr_time_min']) for leg in path)
        
        # Check for duplicates, if yes skip the path
        if node in self.paths:
            existing_keys = [entry[2] for entry in self.paths[node]] # list of each already present path's key 
            if key_new in existing_keys:
                return 

        # Get confidence level, if too low, skip the path
        confidence = self.confidence_fn(path, model=self.delay_model, weather_condition=self.weather_condition, date_of_the_day = self.date_of_the_day)
        if confidence < self.Q:
            return  

        # Get score
        score = self.score_fn(path)
        # Add path to list of path for given node
        if node not in self.paths:
            self.paths[node] = [(score, path, key_new, confidence)]
        else:
            self.paths[node].append((score, path, key_new, confidence))
            # Keep only top-k (sorted by score descending)
            self.paths[node] = sorted(self.paths[node], key=lambda x: -x[0])[:self.k]

            
    '''
    Given a node, get the k top paths
    '''
    def get_k_top_paths(self, node):
        return [(p, conf) for (_, p, _, conf) in self.paths.get(node, [])]


## 5.2 - Algorithm 

In [37]:
'''
Routing algorithm : reverse search algorithm to find top-k paths from a start node to a target node.
    - Explores the graph backwards from the target to compute valid paths arriving before latest_arrival_time and starting at start_node.
    - Keeps at most k best paths per node (best is based on latest departure time).
    - Filters paths using a confidence function (confidence is given by the pretrained ML model).
    - Limits exploration to expand_max expansions per node to control runtime.

 Returns:
    TopKPathsPerNode object with robust path options to each node.
'''
def reverse_routing_algo_top_k(G_rev, start_stop, target_stop, latest_arrival_time, score_fn, confidence_fn,  Q, delay_model, weather_condition, k=5, expand_max=10):
    
    latest_arrival_time_min = time_to_minutes(latest_arrival_time)
    
    top_k_paths = TopKPathsPerNode(k, score_fn, confidence_fn, Q, delay_model=delay_model, 
                                   weather_condition=weather_condition, date_of_the_day=latest_arrival_time)

    queue = PriorityQueue()
    count_u = {node: 0 for node in G_rev.nodes}
    tie_breaker = count()  # to avoid error when 2 dep_time are equal, because cannot compare path dictionnaries, so need a tie breaker

    # Each item in the queue: (neg_time, tie breaker,  current_stop, path_so_far)
    # Start from target with an empty path
    queue.put((-latest_arrival_time_min, next(tie_breaker), target_stop, []))

    while not queue.empty():
        current_time_neg, _, current_stop, path_so_far = queue.get()
        current_time = -current_time_neg

        if count_u[current_stop] >= expand_max:
            continue  # Skip node if already expanded k times

        # We only interested in paths that start at start_stop
        if current_stop == start_stop:
            top_k_paths.add_path(current_stop, path_so_far)
        
        count_u[current_stop] += 1

        for _, neighbor, key, edge_data in G_rev.out_edges(current_stop, keys=True, data=True):
            edge_type = edge_data.get('edge_type', 'transit')

            if edge_type == 'transit':
                dep_time = edge_data['dep_time_min']
                arr_time = edge_data['arr_time_min']
                if arr_time <= current_time:
                    leg = {
                        'from': neighbor,
                        'to': current_stop,
                        'dep_time_min': dep_time,
                        'arr_time_min': arr_time,
                        'trip_id': edge_data.get('trip_id'),
                        'type': 'transit'
                    }
                    new_path = path_so_far + [leg]
                    queue.put((-dep_time, next(tie_breaker), neighbor, new_path))

            elif edge_type in ['transfer', 'distance']:
                duration = edge_data.get('min_transfer_time') or edge_data.get('duration') or 2
                dep_time = current_time - duration
                leg = {
                    'from': neighbor,
                    'to': current_stop,
                    'dep_time_min': dep_time,
                    'arr_time_min': current_time,
                    'trip_id': None,
                    'type': edge_type
                }
                new_path = path_so_far + [leg]
                queue.put((-dep_time, next(tie_breaker), neighbor, new_path))

    return top_k_paths.get_k_top_paths(node=start_stop)

In [38]:
""" Format dep/arr times in minutes into HH:MM strings """
def minutes_to_time_str(minutes):
    if pd.isna(minutes) or minutes == float('-inf') or minutes == float('inf'):
        return ""
    h = int(minutes) // 60
    m = int(minutes) % 60
    return f"{h:02}:{m:02}"


def time_to_minutes(t):
    return t.hour * 60 + t.minute

# 6 - Visualization

## 6.1 - Helper methods

In [39]:
def create_route_df(path):
    """
    Converts a routing path into a DataFrame of readable segments.
    Each row represents a movement (leg) from one stop to the next.
    
    Parameters:
    path (list of dict): A list representing a sequence of legs (from reverse_routing_algo_top_k).
    
    Returns:
    pd.DataFrame: A structured DataFrame of the route.
    """
    df_list = []

    for i, step in enumerate(reversed(path)):  # reverse to show forward direction
        leg_type = step['type']
        dep_min = step["dep_time_min"]
        arr_min = step["arr_time_min"]
        duration = arr_min - dep_min if "duration" not in step else step["duration"]
        trip_id = step.get("trip_id", "")

        df_list.append({
            "step": i + 1,
            "leg_type": leg_type,
            "from_stop_id": step["from"],
            "to_stop_id": step["to"],
            "dep_time": minutes_to_time_str(dep_min),
            "arr_time": minutes_to_time_str(arr_min),
            "duration": duration,
            "trip_id": trip_id,
        })

    return pd.DataFrame(df_list)

In [40]:
def prepare_merges(df):
    # Merge with trip_names_df (assumed to contain 'trip_id', 'route_desc', 'leg_type') 
    df = df.merge(trip_names_df, on='trip_id', how='left')
    
    # First merge to get info for from_stop_id
    df_merged = df.merge(sbb_stops_region, left_on='from_stop_id', right_on='stop_id_cleaned', how='left') \
    .rename(columns={
        'stop_name': 'from_stop_name',
        'stop_lat': 'from_lat',
        'stop_lon': 'from_lon'
    }).drop(columns=['stop_id_cleaned'])
    
    # Second merge to get info for to_stop_id
    df_merged = df_merged.merge(sbb_stops_region, left_on='to_stop_id', right_on='stop_id_cleaned', how='left') \
    .rename(columns={
        'stop_name': 'to_stop_name',
        'stop_lat': 'to_lat',
        'stop_lon': 'to_lon'
    }).drop(columns=['stop_id_cleaned'])

    df_merged['route_short_name'] = df_merged['route_short_name'].fillna('walk')

    return df_merged

In [41]:
def add_emojis(df):
    """
    Input: a DataFrame from create_route_df() and prepare_merges().
    Adds two columns:
        - 'emoji': a visual indicator of the transport type
        - 'transport_name': a readable label for the mode
    """

    transport_emojis = {
        "walk": ("🚶🏽‍♀️", "Walk"),
        "transfer": ("🔁", "Direct Transfer"),
        "B": ("🚌", "Bus"),
        "M": ("Ⓜ️", "Metro"),
        "RE": ("🚉", "RegioExpress"),
        "IR": ("🚄", "InterRegio"),
        "IC": ("🚅", "InterCity"),
        "R": ("🚉", "Regio")
    }

    # Define a function to extract emoji and name
    def get_emoji_name(row):
        key = row.get('route_desc') or row.get('leg_type')
        if key in transport_emojis:
            return transport_emojis[key]
        elif row.get('leg_type') in transport_emojis:
            return transport_emojis[row['leg_type']]
        else:
            return ("❓", key if key else "Unknown")

    # Apply row-wise
    df[['emoji', 'transport_name']] = df.apply(lambda row: pd.Series(get_emoji_name(row)), axis=1)

    return df

## 6.2 - Visualization interface 

In [44]:
from ipywidgets import Dropdown, IntSlider, Button, HBox, VBox, Output, Layout
from IPython.display import display
import plotly.express as px
import plotly.graph_objects as go
import datetime

# Create widgets
start_stop_dd   = Dropdown(options=unique_stop_names, description='Start stop')
end_stop_dd     = Dropdown(options=unique_stop_names, description='End stop')
arrival_hour_sl = IntSlider(min=0, max=23, value=18, description='Arrival hour')
arrival_min_sl  = IntSlider(min=0, max=59, value=0,  description='Arrival min')
weather_dd      = Dropdown(options=['Clear', 'Rain'], description='Weather')
Q_sl            = IntSlider(
    min=0, max=100, value=80, step=1,
    description='Min confidence (%)',
    style={'description_width': '200px'},
    layout=Layout(width='400px')
)
k_sl            = IntSlider(
    min=1, max=10, value=1,
    description='How many possible paths?',
    style={'description_width': '200px'},
    layout=Layout(width='400px')
)
path_ix_sl      = IntSlider(
    min=0, max=9, value=0,
    description='Show path #',
    style={'description_width': '200px'},
    layout=Layout(width='400px')
)

# Create Button and Output area
submit_btn = Button(description='Submit', button_style='primary')
out        = Output()

# Define callback for the button
def on_submit(b):
    out.clear_output()
    with out:
        print("Working…")

        start_stop = start_stop_dd.value
        end_stop   = end_stop_dd.value

        if start_stop == end_stop:
            out.clear_output()
            print("❌ The start and end stop must be different.")
            print("Done")
            return

        arrival_hour      = arrival_hour_sl.value
        arrival_minute    = arrival_min_sl.value
        weather_condition = weather_dd.value
        Q                 = Q_sl.value
        k                 = k_sl.value
        path_index        = path_ix_sl.value

        if path_index >= k:
            out.clear_output()
            print(f"❌ You asked to show path #{path_index}, but you only requested {k} paths.")
            print("Done")
            return

        # Convert stop names to IDs
        start_stop_id = sbb_stops_region.loc[
            sbb_stops_region['stop_name'] == start_stop, 'stop_id_cleaned'
        ].iloc[0]
        end_stop_id = sbb_stops_region.loc[
            sbb_stops_region['stop_name'] == end_stop, 'stop_id_cleaned'
        ].iloc[0]

        # Build arrival_time
        arrival_time = datetime.datetime(
            2025, 5, 14, arrival_hour, arrival_minute
        )

        # Run reverse routing
        top_k_paths = reverse_routing_algo_top_k(
            G,
            start_stop=start_stop_id,
            target_stop=end_stop_id,
            latest_arrival_time=arrival_time,
            score_fn=score_path_by_latest_departure,
            confidence_fn=get_confidence,
            Q=Q/100,
            delay_model=delay_model,
            weather_condition=weather_condition,
            k=k
        )

        if not top_k_paths:
            out.clear_output()
            print("❌ No path found with sufficient confidence.")
            print("Done")
            return

        num_found = len(top_k_paths)
        warning_text = None
        if num_found < k:
            warning_text = f"⚠️ Only {num_found} paths found (requested {k})."

        if path_index >= num_found:
            out.clear_output()
            print(f"❌ Only {num_found} paths found.")
            print("Done")
            return

        # Build & display the chosen path
        path_df = create_route_df(top_k_paths[path_index][0])
        path_df = prepare_merges(path_df)
        path_df = add_emojis(path_df)

        out.clear_output()
        if warning_text:
            print(warning_text)
        print("Done")

        # Plot departure points
        fig = px.scatter_mapbox(
            path_df,
            lat='from_lat',
            lon='from_lon',
            hover_name='from_stop_name',
            hover_data={
                'trip_id': True,
                'leg_type': True,
                'dep_time': True,
                'arr_time': True,
                'transport_name': True
            },
            zoom=11,
            mapbox_style='carto-positron',
            title=(
                f"Route from {start_stop} to {end_stop} "
                f"(arrive by {arrival_hour:02}:{arrival_minute:02})"
            ),
            width=1200,
            height=700 
        )

        # Draw edges
        shown_routes = set()
        for _, row in path_df.iterrows():
            route_name = row['route_short_name']
            color = 'royalblue' if row['leg_type']=='transit' else 'orange'
            hover_text = (
                f"{row['from_stop_name']} → {row['to_stop_name']}<br>"
                f"Mode: {row['transport_name']} {row['emoji']}<br>"
                f"Line: {route_name}<br>"
                f"Trip ID: {row['trip_id'] or '—'}<br>"
                f"Dep: {row['dep_time']} | Arr: {row['arr_time']}<br>"
            )
            show_legend = route_name not in shown_routes
            if show_legend:
                shown_routes.add(route_name)

            fig.add_trace(go.Scattermapbox(
                lat=[row['from_lat'], row['to_lat']],
                lon=[row['from_lon'], row['to_lon']],
                mode='lines',
                line=dict(width=4, color=color),
                hoverinfo='text',
                text=[hover_text],
                name=route_name if show_legend else None,
                showlegend=show_legend
            ))

        # End marker
        last_row = path_df.iloc[-1]
        fig.add_trace(go.Scattermapbox(
            lat=[last_row['to_lat']],
            lon=[last_row['to_lon']],
            mode='markers',
            marker=dict(size=12, color='red'),
            name='End',
            hovertext=f"End: {last_row['to_stop_name']}",
            hoverinfo='text'
        ))

        display(fig)


# Wire up the callback
submit_btn.on_click(on_submit)

# Layout and display
controls = VBox([
    HBox([start_stop_dd, end_stop_dd]),
    HBox([arrival_hour_sl, arrival_min_sl]),
    weather_dd,
    Q_sl,
    k_sl,
    path_ix_sl,
    submit_btn
])
display(controls, out)

interactive(children=(Dropdown(description='Start stop', options=('Bussigny, Buyère', 'Bussigny, Cocagne', 'Bu…